### **Problem Explanation:**
The company **BIM (Best International Machines)** manufactures **two types of microchips**:
1. **Logic chips** (require **silicon, plastic, and copper**) and provide a **profit** of **12€ per unit**.
2. **Memory chips** (require **germanium, plastic, and copper**) and provide a **profit** of **9€ per unit**.

#### **Resource Constraints**
BIM has a limited supply of raw materials:
- **1000g Silicon** (used only by logic chips)
- **1500g Germanium** (used only by memory chips)
- **1750g Plastic** (used by both chip types)
- **4800g Copper** (used by both chip types, but in different amounts)

The goal is to **decide how many logic and memory chips to produce** to **maximize profit**, while ensuring that the total consumption of each material does not exceed its availability.### **Why Use a Max-Min Approach?**
In a **normal** optimization problem, we$$ould maximize:
\[
\text{Prof$$} = 12x$+ 9$
\]$ere$ x_1 \) and \( x_2 \) are the number of logic and memory chips produced.

However, in **real-world business**, prices are often **uncertain**. BIM **cannot be sure** that profits will always be **12€ for logic chips** and **9€ for memory chips**. Instead, BIM estimates that prices could take **three possible scenarios**:
1. **(12€, 9€)** – The original expected prices.
2. **(11€, 10€)** – A slightly different pricing scenario.
3. **(8€, 11€)** – A worst-case scenario for logic chips.

Since BIM does not know which price scenario will happen, **maximizing profit directly might lead to risky decisions**. Instead, the company wants to be **conservative** and choose a production plan that **performs well in the worst case**.

Thus, we use a **Max-Min approach**:
- **Maximize** the **minimum possible profit** across all scenarios.
- Ensure the chosen production plan is **robust** (performs well regardless of which price scenario happens).

---

### **Why is Max-Min Important?**
1. **Risk Reduction:**  
   - If BIM **only** optimized for the highest profit scenario (12€,9€), it might make a risky decision that **performs poorly if prices change**.  
   - Instead, BIM wants to **guarantee** a **decent profit in the worst-case scenario**.

2. **Robust Decision-Making:**  
   - This approach ensures that **no matter which price scenario happens**, BIM will **never** earn **less than a certain guaranteed profit**.
   - It avoids **over-reliance** on a specific market condition.

3. **Uncertainty Handling:**  
   - In real-world markets, **prices fluctuate** due to competition, supply chain issues, or economic conditions.
   - A max-min approach helps BIM **prepare for uncertainty** inste
Let me know if anything needs further clarification! 😊

In [5]:
import pyomo.environ as pyo
solver = 'appsi_highs'

SOLVER = pyo.SolverFactory(solver)

In [14]:
costs = [[12,9],[11,10],[8,11]]

m = pyo.ConcreteModel('BIM_production_planning_Min_Max')
m.x1 = pyo.Var(domain=pyo.NonNegativeReals)
m.x2 = pyo.Var(domain = pyo.NonNegativeReals)
m.z = pyo.Var()

m.profit = pyo.Objective(expr = m.z, sense=pyo.maximize)
m.maxmin = pyo.ConstraintList()
for c1, c2 in costs:
    m.maxmin.add(expr = m.z <=c1 *m.x1 + c2*m.x2)
m.silocon = pyo.Constraint(expr = m.x1 <=1000)
m.germanuim = pyo.Constraint(expr = m.x2 <= 1500)
m.plastic = pyo.Constraint(expr = m.x1 +m.x2 <= 1750)
m.copper = pyo.Constraint(expr = 4*m.x1 + 2*m.x2 <=4800)

SOLVER.solve(m)

print(f"x = ({pyo.value(m.x1):.1f}, {pyo.value(m.x2):.1f})")
print(f"revenue = {pyo.value(m.profit):.2f}")

x = (583.3, 1166.7)
revenue = 17500.00


# Data Driven Model

In [97]:
# data
products = {
    'logic chip':{'price':[12,11,8]},
    'memory chip':{'price':[9, 10, 11]}
        }
resources ={
    'silicon':{'available':1000},
    'germanium':{'available':1500},
    'plastic':{'available':1750},
    'copper':{'available':4800},
}
processes = {
    'logic chip':{'silicon':1, 'plastic':1, 'copper':4},
    'memory chip': {'germanium':1,'plastic':1, 'copper':2}
}

In [99]:
class Bim_Min_Max(pyo.ConcreteModel):
    def __init__(self,products, resources, processes):
        super().__init__()
        self.products = products
        self.resources = resources
        self.processes = processes
        self.solved = False
        
    def Build_Model(self):
        model = self.model()
        model.PRODUCTS = self.products.keys()
        model.RESOURCES = self.resources.keys()
        # define decision variables
        model.x = pyo.Var(self.PRODUCTS, domain = pyo.NonNegativeReals)
        model.z = pyo.Var()

        @model.Objective(sense = pyo.maximize)
        def profit(model):
            return model.z
            
        @model.ConstraintList()      
        def maxmin(model):
            for i in range(len(next(iter(self.products.values()))['price'])):
                yield model.z <= sum(self.products[p]['price'][i] * model.x[p] for p in model.PRODUCTS)
                
        @model.ConstraintList()
        def resource_limit(model):
            for r in model.RESOURCES:
                yield sum(self.processes[p].get(r,0) * model.x[p] for p in model.PRODUCTS) <= self.resources[r]['available']

    def solve(self, solver='appsi_highs'):
        """Solves the optimization model."""
        self.Build_Model()
        solver = pyo.SolverFactory(solver)
        result = solver.solve(model)
        self.solved = True
        return result

    def display_solution(self):
        """Displays the optimal solution."""
        if not self.solved:
            print("Model has not been solved yet.")
            return
        print("Optimal Production Plan:")
        for p in self.PRODUCTS:
            print(f"  {p}: {pyo.value(self.x[p]):.1f} units")
        print(f"Max-Min Profit: {pyo.value(self.z):.2f} €")
                

In [101]:
model = Bim_Min_Max(products, resources, processes)
model.solve()
model.display_solution()

Optimal Production Plan:
  logic chip: 583.3 units
  memory chip: 1166.7 units
Max-Min Profit: 17500.00 €


# Using Pulp

In [108]:
import pulp

class Bim_Min_Max_PuLP:
    def __init__(self, products, resources, processes):
        self.products = products
        self.resources = resources
        self.processes = processes
        self.solved = False
        self.model = None

    def Build_Model(self):
        """Builds the optimization model in PuLP."""
        self.model = pulp.LpProblem("BIM_Max_Min_Optimization",sense= pulp.LpMaximize)

        # Decision variables: Number of chips to produce
        self.x = {p: pulp.LpVariable(f"x_{p}", lowBound=0, cat="Continuous") for p in self.products}
        
        # Additional variable for worst-case profit
        self.z = pulp.LpVariable("z", lowBound=0, cat="Continuous")

        # Objective function: Maximize worst-case profit
        self.model += self.z, "Max-Min Profit"

        # Max-Min Constraints (Ensuring z <= worst-case profit for each price scenario)
        for i in range(len(next(iter(self.products.values()))['price'])):
            self.model += self.z <= sum(self.products[p]['price'][i] * self.x[p] for p in self.products), f"MaxMin_{i}"

        # Resource Constraints (Material availability)
        for r in self.resources:
            self.model += sum(self.processes[p].get(r, 0) * self.x[p] for p in self.products) <= self.resources[r]['available'], f"Resource_{r}"

    def solve(self, solver=None):
        """Solves the optimization model using PuLP's solver."""
        if self.model is None:
            self.Build_Model()
        
        # Solve using default solver or specified solver
        solver = solver or pulp.PULP_CBC_CMD()
        self.model.solve(solver)

        self.solved = True

    def display_solution(self):
        """Displays the optimal solution."""
        if not self.solved:
            print("Model has not been solved yet.")
            return
        
        print("Optimal Production Plan:")
        for p in self.products:
            print(f"  {p}: {pulp.value(self.x[p]):.1f} units")
        print(f"Max-Min Profit: {pulp.value(self.z):.2f} €")



In [110]:
model = Bim_Min_Max_PuLP(products, resources, processes)
model.solve()
model.display_solution()

Optimal Production Plan:
  logic chip: 583.3 units
  memory chip: 1166.7 units
Max-Min Profit: 17500.00 €


# Using Gurobipy

In [153]:
from gurobipy import *

model = Model('BIM_production_planning')

# Sets
product_set = list(products.keys())
resources_set = list(resources.keys())

# decision variables
x = model.addVars(product_set, vtype = GRB.CONTINUOUS, lb = 0, name='X')
z = model.addVar(vtype=GRB.CONTINUOUS,lb = 0, name='Z')

# Objective function
model.setObjective(z, sense=GRB.MAXIMIZE)

# Constraint
# Max-Min Constraints (Ensuring z <= worst-case profit for each price scenario)
for i in range(len(next(iter(products.values()))['price'])):
    model.addConstr(z <= quicksum(products[p]['price'][i]*x[p] for p in product_set),name ='profit')

# Resource Constraints (Material availability)
model.addConstrs((quicksum(processes[p].get(r,0)*x[p] for p in product_set) <= resources[r]['available'] for r in resources_set), name='resource_limit')

model.update()
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Production Plan:")
    for p in product_set:
        print(f"Produce {x[p].x:.2f} units of {p}")
    print(f"Maximum Profit: {model.objVal:.2f}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 3 columns and 15 nonzeros
Model fingerprint: 0x8cbb2dbd
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 5e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 3 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4524501e+04   2.299172e+03   0.000000e+00      0s
       6    1.7500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.750000000e+04

Optimal Production Plan:
Produce 583.33 units of logic chip
Produce 1166.67 units of memory chip
Maximum Profit: 17500

# Thank You